#### Importando bibliotecas

In [1]:
from pymongo import MongoClient
import requests
import pandas as pd

#### Conectando ao MongoDB Atlas

In [ ]:
# Conectar ao MongoDB Atlas
client = MongoClient("")
# Seleciona o banco de dados
db = client[""]
# Seleciona a coleção
collection = db["meios_pagamentos_trimestrais"]

In [3]:
# Recuperar dados da coleção
data = list(collection.find())
data

[]

In [4]:
# Converter para DataFrame
df = pd.DataFrame(data)
print(df)

Empty DataFrame
Columns: []
Index: []


#### Consumindo API e enviando dados para o banco

In [5]:
# Endpoint da nova API para os dados de meios de pagamento trimestrais
url = ("https://olinda.bcb.gov.br/olinda/servico/MPV_DadosAbertos/versao/v1/odata/MeiosdePagamentosTrimestralDA(trimestre=@trimestre)?@trimestre=%2720201%27&$top=100&$format=json&$select=datatrimestre,valorPix,valorTED,valorTEC,valorCheque,valorBoleto,valorDOC,valorCartaoCredito,valorCartaoDebito,valorCartaoPrePago,valorTransIntrabancaria,valorConvenios,valorDebitoDireto,valorSaques,quantidadePix,quantidadeTED,quantidadeTEC,quantidadeCheque,quantidadeBoleto,quantidadeDOC,quantidadeCartaoCredito,quantidadeCartaoDebito,quantidadeCartaoPrePago,quantidadeTransIntrabancaria,quantidadeConvenios,quantidadeDebitoDireto,quantidadeSaques")

# Realizar a requisição
response = requests.get(url)

# Verificar se a requisição foi bem-sucedida
if response.status_code == 200:
    data = response.json()

    # Inicializar a lista para armazenar os dados
    pagamento_data = []

    # Iterar pelos resultados
    for item in data["value"]:

        # Extrair o ano e o trimestre
        datatrimestre = item["datatrimestre"]
        ano = datatrimestre[:4]
        trimestre = int(datatrimestre[5:7])  # Os trimestres são no formato "YYYY-MM", portanto pegamos o mês

        # Determinar o nome do trimestre
        if trimestre == 3:
            nome_trimestre = "1º Trimestre"
        elif trimestre == 6:
            nome_trimestre = "2º Trimestre"
        elif trimestre == 9:
            nome_trimestre = "3º Trimestre"
        else:
            nome_trimestre = "4º Trimestre"

        # Categorias de pagamento
        categorias = [
            "Pix", "TED", "TEC", "Cheque", "Boleto", "DOC",
            "CartaoCredito", "CartaoDebito", "CartaoPrePago",
            "TransIntrabancaria", "Convenios", "DebitoDireto", "Saques"
        ]

        # Adicionar os dados de cada categoria
        for categoria in categorias:
            pagamento_data.append({
                "ano": ano,
                "trimestre": nome_trimestre,
                "metodo": categoria,
                "quantidade": item[f"quantidade{categoria}"],
                "valor": item[f"valor{categoria}"]
            })

else:
    print("Erro:", response.status_code, response.text)


In [6]:
# Converter para DataFrame
df = pd.DataFrame(pagamento_data)
df.head()

,ano,trimestre,metodo,quantidade,valor
0,2024,2º Trimestre,Pix,15433584.90,6403013.64
1,2024,2º Trimestre,TED,203269.58,10662555.73
2,2024,2º Trimestre,TEC,0.00,0.00
3,2024,2º Trimestre,Cheque,44981.97,221553.82
4,2024,2º Trimestre,Boleto,1439104.18,2360420.40


In [7]:
#tranformar dados em dicionário para insertir no banco de dados
data_dict = df.to_dict("records")
print(data_dict)

[{'ano': '2024', 'trimestre': '2º Trimestre', 'metodo': 'Pix', 'quantidade': 15433584.9, 'valor': 6403013.64}, {'ano': '2024', 'trimestre': '2º Trimestre', 'metodo': 'TED', 'quantidade': 203269.58, 'valor': 10662555.73}, {'ano': '2024', 'trimestre': '2º Trimestre', 'metodo': 'TEC', 'quantidade': 0.0, 'valor': 0.0}, {'ano': '2024', 'trimestre': '2º Trimestre', 'metodo': 'Cheque', 'quantidade': 44981.97, 'valor': 221553.82}, {'ano': '2024', 'trimestre': '2º Trimestre', 'metodo': 'Boleto', 'quantidade': 1439104.18, 'valor': 2360420.4}, {'ano': '2024', 'trimestre': '2º Trimestre', 'metodo': 'DOC', 'quantidade': 0.0, 'valor': 0.0}, {'ano': '2024', 'trimestre': '2º Trimestre', 'metodo': 'CartaoCredito', 'quantidade': 4879269.34, 'valor': 631789.1}, {'ano': '2024', 'trimestre': '2º Trimestre', 'metodo': 'CartaoDebito', 'quantidade': 4057958.85, 'valor': 239101.56}, {'ano': '2024', 'trimestre': '2º Trimestre', 'metodo': 'CartaoPrePago', 'quantidade': 3046202.02, 'valor': 77386.79}, {'ano': '20

In [8]:
# Inserindo os dados no MongoDB
collection.insert_many(data_dict)

InsertManyResult([ObjectId('6757174d8b005471faa09df1'), ObjectId('6757174d8b005471faa09df2'), ObjectId('6757174d8b005471faa09df3'), ObjectId('6757174d8b005471faa09df4'), ObjectId('6757174d8b005471faa09df5'), ObjectId('6757174d8b005471faa09df6'), ObjectId('6757174d8b005471faa09df7'), ObjectId('6757174d8b005471faa09df8'), ObjectId('6757174d8b005471faa09df9'), ObjectId('6757174d8b005471faa09dfa'), ObjectId('6757174d8b005471faa09dfb'), ObjectId('6757174d8b005471faa09dfc'), ObjectId('6757174d8b005471faa09dfd'), ObjectId('6757174d8b005471faa09dfe'), ObjectId('6757174d8b005471faa09dff'), ObjectId('6757174d8b005471faa09e00'), ObjectId('6757174d8b005471faa09e01'), ObjectId('6757174d8b005471faa09e02'), ObjectId('6757174d8b005471faa09e03'), ObjectId('6757174d8b005471faa09e04'), ObjectId('6757174d8b005471faa09e05'), ObjectId('6757174d8b005471faa09e06'), ObjectId('6757174d8b005471faa09e07'), ObjectId('6757174d8b005471faa09e08'), ObjectId('6757174d8b005471faa09e09'), ObjectId('6757174d8b005471faa09e